# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [14]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score

## Загрузка и подготовка данных

In [23]:
df0 = pd.read_csv('/datasets/geo_data_0.csv', sep=',')
df1 = pd.read_csv('/datasets/geo_data_1.csv', sep=',')
df2 = pd.read_csv('/datasets/geo_data_2.csv', sep=',')

In [42]:
def intro(x):
    print('\033[1m' + 'Общая информация:')
    print ('\033[0m')
    display(x.info())
    print('\033[1m' + 'Первые пять строк:')
    print ('\033[0m')
    display(x.head())
    print('\033[1m' + 'Последние пять строк:')
    print ('\033[0m')
    display(x.tail())
    print('\033[1m' + 'Описание данных:')
    print ('\033[0m')
    display(x.describe())  
    print('\033[1m' + 'Дубликаты:')
    print ('\033[0m')
    display(x.duplicated().sum())

In [25]:
intro(df0)

Общая информация:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Первые пять строк:



,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Последние пять строк:



,id,f0,f1,f2,product
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764
99999,1CWhH,1.764754,-0.266417,5.722849,149.633246


Описание данных:



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


Дубликаты:



0

In [26]:
intro(df1)

Общая информация:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Первые пять строк:



,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Последние пять строк:



,id,f0,f1,f2,product
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364
99999,relB0,-3.426139,-7.794274,-0.003299,3.179103


Описание данных:



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


Дубликаты:



0

In [27]:
intro(df2)

Общая информация:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Первые пять строк:



,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Последние пять строк:



,id,f0,f1,f2,product
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253
99999,V9kWn,-2.551421,-2.025625,6.090891,102.775767


Описание данных:



,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Дубликаты:



0

Условия задачи:
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

Функия которая подготовит данные для всех трех датасетов :

In [28]:
def stage1(df):
    features = df.drop(['id','product'], axis=1) 
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=180495) 
    print('Размер обучающей выборки:', features_train.shape)
    print('Разр валидационной выборки:', features_valid.shape)
    model = LinearRegression() 
    model.fit(features_train,target_train)
    target_valid = target_valid.reset_index(drop=True)
    predictions = pd.Series(model.predict(features_valid), index=target_valid.index)
    
    print(('Cредний запас сырья: {:.4f}'.format(predictions.mean())))
    print('RMSE: {:.4f}'.format(mean_squared_error(target_valid,predictions)**0.5))
    return target_valid, predictions

## Обучение и проверка модели

Применим функцию к данным 3 х регионов:

**Регион 1**

In [29]:
target_1, predict_1 = stage1(df0)

Размер обучающей выборки: (75000, 3)
Разр валидационной выборки: (25000, 3)
Cредний запас сырья: 92.5319
RMSE: 37.7200


**Регион 2**

In [30]:
target_2, predict_2 = stage1(df1)

Размер обучающей выборки: (75000, 3)
Разр валидационной выборки: (25000, 3)
Cредний запас сырья: 68.9406
RMSE: 0.8867


**Регион 3**

In [31]:
target_3, predict_3 = stage1(df2)

Размер обучающей выборки: (75000, 3)
Разр валидационной выборки: (25000, 3)
Cредний запас сырья: 94.8259
RMSE: 40.0086


**Выводы :**  
В превом и третьем регионе достаточно высокие показатели по среднему объему скважин, однако по RMSE видно что модель ошибается почти на половину от предсказанного что, во втором же регионе модель предсказала самый низкий показатель сырья но у модели хорошая точность.

## Подготовка к расчёту прибыли

Нужно рассчитать объём сырья для безыбыточной разаработки одной скважины:
* budget - бюджет на разработку 
* income_b - с доход сединицы продукта(1000 барелей)
* count - количество скважин

Переведем все в тсячи чтоб было удобно. Что бы посчитать минимальный объем с одной скважины поделим бюджет на произведение стоимости бареля и количество скважин(предполагается 200):

In [43]:
BUDGET = 10_000_000_000
INCOME_B = 450
COUNT = 200
volume_min = BUDGET/(COUNT*INCOME_B)
print('Минимальный объем соскважины для безубыточной разработки:{:.4f}'.format(volume_min))
print('Средний объём скважины в "Регоионе 1" : {:.4f}'.format(df0['product'].mean()))
print('Средний объём скважины в "Регоионе 2" : {:.4f}'.format(df1['product'].mean()))
print('Средний объём скважины в "Регоионе 3" : {:.4f}'.format(df2['product'].mean()))

Минимальный объем соскважины для безубыточной разработки:111111.1111
Средний объём скважины в "Регоионе 1" : 92.5000
Средний объём скважины в "Регоионе 2" : 68.8250
Средний объём скважины в "Регоионе 3" : 95.0000


**Выводы :** 

Посчитан показатель минимального объема в скважине при выборе 200 скважин.
Данный показатель выше среднего объема скважин во всех трех регионах, 111.1111 тыс.бар.

Для рассчета  прибыли нам нужно умножить суммарный  объем скважин на стоимость одной единицы продукта затем вычесть бюджет на разработку.

## Расчёт прибыли и рисков 

По условию задачи нам нужно выбрать случайно 500 скважин из них взять 200 лучших,
посчитать объём и затем посчитать прибыль

In [33]:
state = 180495

def stage2(target, predict, count):
    sample_predict = predict.sample(n=500, replace = False, random_state=state)
    predict_hi = predict.sort_values(ascending=False)[:count]  
    target_hi = target[predict_hi.index]
    volume = sum(target_hi)
    income = volume*income_b - budget
    print('Средний запас сырья среди выбранных скважин: {:.2f}'.format(target_hi.mean()))
    print('Суммарный объём: {:.2f}'.format(volume))
    print('Прибыль : {:.2f} тыс.руб.'.format(income))
    return(income)

**Регион 1**

In [34]:
stage2(target_1,predict_1,count)

Средний запас сырья среди выбранных скважин: 149.74
Суммарный объём: 29947.77
Прибыль : 3476494.71 тыс.руб.


3476494.7091399767

**Регион 2**

In [35]:
stage2(target_2,predict_2,count)

Средний запас сырья среди выбранных скважин: 137.95
Суммарный объём: 27589.08
Прибыль : 2415086.70 тыс.руб.


2415086.6966815516

**Регион 3**

In [36]:
stage2(target_3,predict_3,count)

Средний запас сырья среди выбранных скважин: 140.50
Суммарный объём: 28100.60
Прибыль : 2645271.66 тыс.руб.


2645271.658534663

Максимальная прибыль с 200 более продуктивных скважин в регионе 1  - 3 476494.71 тыс.руб.

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль

Напишем функцию что бы потом применить к трём регионам:

Напишем функцию повторяющую предыдущую которая будет возвращать доход:

In [37]:
def revenue(target, predict, count):
    predict_hi = predict.sort_values(ascending=False)  
    target_hi = target[predict_hi.index][:count]
    volume = sum(target_hi)
    income = volume*income_b - budget
    return income

Используя функцию revenue напишем фунцию которя применит технику Bootstrap с 1000 почитает среднюю прибыль, 
доверительный интервал и риски:

In [38]:
def stage3(target, predict, count):
    state = np.random.RandomState(180495)
    bootstrap_samples = 1000
    values = []
    for i in range(bootstrap_samples):
        target_subsample = target.sample(n=500,replace = True, random_state=state)
        predict_subsample = predict[target_subsample.index]
        values.append(revenue(target_subsample,predict_subsample,count))

    values = pd.Series(values)
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    
    damage = 0
    for n in values:
        if n < 0 :
            damage += 1
    damages = damage / bootstrap_samples * 100
    
    print('Средняя прибыль {:.2f} тыс.руб.'.format(values.mean()))
    print('95% доверительный интервал: {:.2f} : {:.2f} '.format(lower,upper))
    print('Риск убытков: {:.2f} %'.format(damages))

**Регион 1**

In [39]:
stage3(target_1,predict_1,count)

Средняя прибыль 520323.11 тыс.руб.
95% доверительный интервал: 3054.96 : 1099361.78 
Риск убытков: 2.50 %


**Регион 2**

In [40]:
stage3(target_2,predict_2,count)

Средняя прибыль 533250.77 тыс.руб.
95% доверительный интервал: 104956.90 : 959983.80 
Риск убытков: 1.00 %


**Регион 3**

In [41]:
stage3(target_3,predict_3,count)

Средняя прибыль 403059.81 тыс.руб.
95% доверительный интервал: -145202.76 : 928171.97 
Риск убытков: 7.10 %


Как видно из полученных результатов допустимому параметру риска, ниже 2.5% отвечает только Регион 2

## Выводы:

На основании проведенного исследования предоставленных данных рекомендую выбрать для разработки учаток №2. При небольших рисках потенциальная прибыль с этого участка, согласно предсказаний модели, самая высокая, а доверительный интервал для средней прибыли с участка значительно "выше" нуля, в отличие от остальных предложенных к рассмотрению. Всё это позволяет с уверенностью утверждать, что разработка этого участка окажется наиболее выгодной для компании.